# Fallbacks(폴백)
##### 언어 모델로 작업할 때 속도 제한이나 가동 중지 시간과 같은 기본 API 문제가 발생할 수 있다.
##### 프로덕션 환경에서 Fallback은 긴급 상황을 대체할 수 있는 좋은 수단이다.

## LLM API 오류
##### LLM API에 대한 요청은 다양한 이유로 실패할 수 있다.(API 다운 or 비율 제한 등등)

##### 먼저 OpenAI에서 RateLimitError가 발생하면 어떤 일이 일어나는지 확인해보자
```python
from unittest.mock import patch

import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")
response = httpx.Response(200, request=request)
error = RateLimitError("rate limit", response=response, body="")
```

```python
# RateLimits 반복을 피하기 위해 max_retries를 0으로 설정한다.
openai_llm = ChatOpenAI(max_retries=0)
anthropic_llm = ChatAnthropic()
llm = openai_llm.with_fallbacks([anthropic_llm])

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(openai_llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

Hit error
```

```python
# Anthropic 으로 대체하는 폴백을 해보자
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

content=' I don\'t actually know why the chicken crossed the road, but here are some possible humorous answers:\n\n- To get to the other side!\n\n- It was too chicken to just stand there. \n\n- It wanted a change of scenery.\n\n- It wanted to show the possum it could be done.\n\n- It was on its way to a poultry farmers\' convention.\n\nThe joke plays on the double meaning of "the other side" - literally crossing the road to the other side, or the "other side" meaning the afterlife. So it\'s an anti-joke, with a silly or unexpected pun as the answer.' additional_kwargs={} example=False
```

##### 또한, 일반 LLM 처럼 폴백이 포함된 LLM을 사용할 수 있다.

```python
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
chain = prompt | llm
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"animal": "kangaroo"}))
    except RateLimitError:
        print("Hit error")
```

## 시퀀스에 대한 대체
##### 시퀀스 자체인 시퀀스에 대한 대체를 만들 수 있다.

```python
# ChatModel을 사용하여 체인을 만들고 문자열 출력 파서를 추가하여 동일한 출력이 되게 하자
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
# 잘못된 모델 이름을 사용하여 오류가 발생하는 체인을 만들어보자
chat_model = ChatOpenAI(model="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()
```

```python
# 평범한 OpenAI 모델 체인을 만들어보자
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)
llm = OpenAI()
good_chain = prompt | llm
```

```python
# 두 모델을 결합한 하나의 모델 체임을 만들 수 있다. bad_chain + good_chain
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

'\n\nAnswer: The turtle crossed the road to get to the other side, and I have to say he had some impressive determination.'
```

## 긴 입력
##### LLM의 가장 큰 제한 요소 중 하나는 컨텍스트 창이다.
##### 일반적으로 프롬프트를 LLM으로 보내기 전에 프롬프트의 길이를 계산하고 추적할 수 있지만, 이것이 어려운 상황이라면 컨텍스트 길이가 더 긴 모델로 대체할 수 있다.

```python
short_llm = ChatOpenAI()
long_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
llm = short_llm.with_fallbacks([long_llm])

inputs = "What is the next number: " + ", ".join(["one", "two"] * 3000)

try:
    print(short_llm.invoke(inputs))
except Exception as e:
    print(e)

This model's maximum context length is 4097 tokens. However, your messages resulted in 12012 tokens. Please reduce the length of the messages.
```

```python
try:
    print(llm.invoke(inputs))
except Exception as e:
    print(e)

content='The next number in the sequence is two.' additional_kwargs={} example=False
```

## 더 나은 모델
##### 모델이 구문 분석에 실패하면 더 나은 모델을 사용하는 로직을 구성할 수 있다.

```python
from langchain.output_parsers import DatetimeOutputParser

prompt = ChatPromptTemplate.from_template(
    "what time was {event} (in %Y-%m-%dT%H:%M:%S.%fZ format - only return this value)"
)

# LLM + 출력 파서 수준에서 폴백을 수행한다. -> 해당 오류는 출력 파서에서 나타나기 때문에
openai_35 = ChatOpenAI() | DatetimeOutputParser()
openai_4 = ChatOpenAI(model="gpt-4") | DatetimeOutputParser()

only_35 = prompt | openai_35
fallback_4 = prompt | openai_35.with_fallbacks([openai_4])

try:
    print(only_35.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

Error: Could not parse datetime string: The Super Bowl in 1994 took place on January 30th at 3:30 PM local time. Converting this to the specified format (%Y-%m-%dT%H:%M:%S.%fZ) results in: 1994-01-30T15:30:00.000Z
```

```python
try:
    print(fallback_4.invoke({"event": "the superbowl in 1994"}))
except Exception as e:
    print(f"Error: {e}")

1994-01-30 15:30:00
```